In [ ]:
import numpy as np
import pandas as pd
import string
import spacy

from matplotlib.pyplot import imread
from matplotlib import pyplot as plt
from wordcloud import WordCloud
%matplotlib inline

In [ ]:
data=pd.read_csv("Elon_musk.csv", encoding = 'unicode_escape')
data

,Unnamed: 0,Text
0,1,@kunalb11 Im an alien
1,2,@ID_AA_Carmack Ray tracing on Cyberpunk with H...
2,3,@joerogan @Spotify Great interview!
3,4,@gtera27 Doge is underestimated
4,5,@teslacn Congratulations Tesla China for amazi...
...,...,...
1994,1995,"@flcnhvy True, it sounds so surreal, but the n..."
1995,1996,@PPathole Make sure to read ur terms &amp; con...
1996,1997,@TeslaGong @PPathole Samwise Gamgee
1997,1998,@PPathole Altho Dumb and Dumber is <U+0001F525...


In [ ]:
data = [Text.strip() for Text in data.Text]
data = [Text for Text in data if Text]
data[0:10]

In [ ]:
text = " ".join(data)
text

In [ ]:
no_punc_text = text.translate(str.maketrans('','',string.punctuation))
no_punc_text

In [ ]:
nlp = spacy.load("en_core_web_sm")

one_block = data[2]
doc_block = nlp(one_block)
spacy.displacy.render(doc_block, style='ent', jupyter=True)

In [ ]:
one_block
for token in doc_block[0:20]:
    print(token, token.pos_)

In [ ]:
nouns_verbs = [token.text for token in doc_block if token.pos_ in ('NOUN', 'VERB')]
print(nouns_verbs[5:25])

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
text_tokens=word_tokenize(no_punc_text)
print(text_tokens[0:50])

In [ ]:
len(text_tokens)
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
my_stop_words = stopwords.words('english')

In [ ]:
my_stop_words

In [ ]:
my_stop_words.append('would')
my_stop_words

In [ ]:
my_stop_words.append('himself')
my_stop_words

In [ ]:
no_stop_tokens = [word for word in text_tokens if not word in my_stop_words]
print(no_stop_tokens[0:50])

In [ ]:
lower_words = [Text.lower() for Text in no_stop_tokens]
print(lower_words[0:50])

In [ ]:
from nltk.stem import PorterStemmer
ps =PorterStemmer()
stemmed_tokens = [ps.stem(word) for word in lower_words]
print(stemmed_tokens[0:50])

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
doc = nlp(' '.join(no_stop_tokens))
print(doc[0:50])

In [ ]:
lemmas = [token.lemma_ for token in doc]
print(lemmas[0:50])

In [ ]:
len(lemmas)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer()
bow_matrix = vectorizer.fit_transform(data)

In [ ]:
print(bow_matrix.toarray())

In [ ]:
bow_matrix.toarray().shape

In [ ]:
vectorizer_ngram = CountVectorizer(analyzer='word',ngram_range=(1,2),max_features = 100)

bow_matrix_ngram = vectorizer_ngram.fit_transform(data)

In [ ]:
print(bow_matrix_ngram.toarray())

In [ ]:
bow_matrix_ngram.toarray().shape

In [ ]:
afinn = pd.read_csv("C:/Users/Rishi/Downloads/Afinn.csv", sep=',', encoding='latin-1')
afinn.shape

In [ ]:
afinn.head()

In [ ]:
from nltk import tokenize
sentences = tokenize.sent_tokenize(' '.join(data))
sentences[5:15]

In [ ]:
sent_df = pd.DataFrame(sentences, columns=['sentence'])
sent_df

In [ ]:
affinity_scores = afinn.set_index('word')['value'].to_dict()

In [ ]:
nlp = spacy.load('en_core_web_sm')
sentiment_lexicon = affinity_scores

def calculate_sentiment(text: str = None):
    sent_score = 0
    if text:
        sentence = nlp(text)
        for word in sentence:
            sent_score += sentiment_lexicon.get(word.lemma_, 0)
    return sent_score

In [ ]:
calculate_sentiment(text = 'amazing')

In [ ]:
sent_df

In [ ]:
sent_df['sentiment_value'] = sent_df['sentence'].apply(calculate_sentiment)

In [ ]:
sent_df['word_count'] = sent_df['sentence'].str.split().apply(len)
sent_df['word_count'].head(10)

In [ ]:
sent_df.sort_values(by='sentiment_value').tail(10)

In [ ]:
sent_df['sentiment_value'].describe()

In [ ]:
sent_df[sent_df['sentiment_value']<=0].head()

In [ ]:
sent_df[sent_df['sentiment_value']>=17].head()

In [ ]:
sent_df['index']=range(0,len(sent_df))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.distplot(sent_df['sentiment_value'])

In [ ]:
plt.figure(figsize=(15, 10))
sns.lineplot(y='sentiment_value',x='index',data=sent_df)

In [ ]:
sent_df.plot.scatter(x='word_count', y='sentiment_value', figsize=(8,8), title='Sentence sentiment value to sentence word count')